In [ ]:
import pandas as pd
from classifier.classifier import Classifier
from classifier.classifier_factory import ClassifierFactory
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler
import pickle

In [ ]:
## initializing classifier_class
classifier_class = ClassifierFactory.build(Classifier.WEEKLY_STOCK_SPECULATION_CLASSIFIER)

In [ ]:
backtest_start_date = datetime(2016,1,1)
backtest_end_date = datetime(2023,1,1)
current_year = datetime.now().year

In [ ]:
## creating database objects
market = Market()
modeler_strat = UniversalModeler()

In [ ]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [ ]:
market.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices(classifier_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = classifier_class.training_set(ticker,prices,False)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
data = pd.concat(training_sets)
training_data = data.dropna().copy().sort_values(["year","week"])

In [ ]:
# classifier_class.db.connect()
# model_start_date = backtest_start_date - timedelta(classifier_class.time_horizon_class.model_date_offset)
# model_end_date = backtest_end_date - timedelta(classifier_class.time_horizon_class.model_date_offset)
# for year in range(model_start_date.year,model_end_date.year):
#     training_slice = training_data[(training_data["year"]<year) & (training_data["year"] >= year - 4)].reset_index(drop=True)
#     prediction_set = training_data[training_data["year"]==year].reset_index(drop=True)
#     stuff = modeler_strat.classification_model(training_slice,prediction_set,classifier_class.factors,False)
#     stuff = stuff.rename(columns={"prediction":f"classification_prediction"})
#     relevant_columns = list(set(list(stuff.columns)) - set(classifier_class.factors))
#     stuff = classifier_class.sim_processor(stuff)
#     classifier_class.db.store("sim",stuff[relevant_columns])
# classifier_class.db.disconnect()

In [ ]:
# classifier_class.db.connect()
# training_slice = training_data[(training_data["year"]<current_year) & (training_data["year"] >= current_year - 4)].reset_index(drop=True)
# stuff = modeler_strat.recommend_classification_model(training_slice,classifier_class.factors,False)
# stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]
# stuff["year"] = current_year
# classifier_class.db.store("models",stuff)
# classifier_class.db.disconnect()

In [ ]:
market.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices(classifier_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = classifier_class.training_set(ticker,prices,True)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
data = pd.concat(training_sets)
training_data = data.dropna().copy().sort_values(["year","week"])

In [ ]:
training_data

In [ ]:
classifier_class.db.connect()
classifier_class.db.drop("predictions")
prediction_set = training_data[training_data["year"]==current_year].reset_index(drop=True)
models = classifier_class.db.retrieve("models")
stuff = modeler_strat.recommend_classification(models,prediction_set,classifier_class.factors)
stuff = stuff.rename(columns={"prediction":f"classification_prediction"})
relevant_columns = list(set(list(stuff.columns)) - set(classifier_class.factors))
stuff = classifier_class.sim_processor(stuff)
classifier_class.db.store("predictions",stuff[relevant_columns])
classifier_class.db.disconnect()

In [ ]:
stuff